# 실험 02-01 다이오드 반파 정류회로

*******************************************************************************
## 공통 코드

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

def tangent_line(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return lambda t: d*t - d*x + f(x)

def get_simulation_result(file_name, start=0, end=-1):
    with open(file_name, encoding='cp1252') as data_file:
        lines = data_file.read()
        occurrences = lines.count('Step Information:')
        data_file.seek(0)

        line = data_file.readline()
        labels = re.split(', | ,|\t', line)
        labels = [s.strip().upper() for s in labels]

        data = {}

        if (occurrences == 0):
            for label in labels:
                data[label] = []
            
            for line in data_file:
                values = re.split(', | ,|\t', line)
                for i in range(len(values)):
                    value = float(values[i]) * 1000
                    data[labels[i]].append(value)

        else:
            labels_all =[]
            
            lines = data_file.readline() # skip first line starting with 'Step Information:'

            for idx in range(occurrences):
                labels_new = []
                for label in labels:
                    labels_new.append('(%s)@%d' % (label, idx+1))
                labels_all += labels_new

                for label_new in labels_new:
                    data[label_new] = []

                for line in data_file:
                    if (line.startswith('Step Information:')):
                        break
                    values = re.split(', | ,|\t', line)
                    for i in range(len(values)):
                        value = float(values[i]) * 1000
                        data[labels_new[i]].append(value)

            labels = labels_all

        for label in labels:
            data[label] = np.array(data[label][start:end])

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def get_oscilloscpoe_result_tektronix(file_name, start=0, end=-1):
    df = pd.read_csv(file_name, header=None, encoding='cp1252',low_memory=False)

    label_ri = df.loc[df.iloc[:,0] == 'Source', 1].index
    ci = 0
    data = {}
    data['TIME'] = df.iloc[start:end, 3].to_numpy().astype(float)
    data['TIME'] -= data['TIME'][0]
    for idx in range(df.shape[1]//6):
        label = df.iloc[label_ri, ci+1].item()
        data[label] = df.iloc[start:end,ci+4].to_numpy().astype(float)
        ci += 6

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def print_array(label, values):
    print('%s = [' % label, end='')
    for idx, vd in enumerate(values):
        print('{:7.3f}'.format(vd), end='')
        if (idx+1 != len(values)):
            print(', ', end='')
    print(']')

def print_value(label, value):
    print('%s = ' % label, end='')
    print('{:7.3f}'.format(value))

def draw_plot(xs, ys, label, style_idx, marker_num=16):
    linestyle  = ['-', '-', '-', '-', '-', '-', '-', '-']
    colors     = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
    markers    = ['o', 'v', 's', 'p', '*', 'h', 'x', 'd']

    markeverys = []
    x_step = (xs.max() - xs.min()) / marker_num
    for idx in range(len(markers)):
        x_start = xs.min() + ((x_step / (len(markers) + 1)) * idx)
        marker_list = []
        for j in range(marker_num):
            tmp_array = np.where(xs > (x_start + (x_step * j)))
            if (len(tmp_array[0]) > 0):
                marker_list.append(tmp_array[0][0])
        markeverys.append(marker_list)

    plt.plot(xs, ys, ls=linestyle[style_idx], color=colors[style_idx], marker=markers[style_idx], markevery=markeverys[style_idx], label=label)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

def set_plot_size(xrate, yrate):
    if (len(set_plot_size.g_plt_figsize) == 0):
        set_plot_size.g_plt_figsize = plt.rcParams["figure.figsize"]
    size = []
    size.append(set_plot_size.g_plt_figsize[0] * xrate)
    size.append(set_plot_size.g_plt_figsize[1] * yrate)
    plt.figure(figsize=size)

set_plot_size.g_plt_figsize = []

*******************************************************************************
## 실험 1 결과 분석 (실험 과정 4)

### 회로도

* 사용한 소자
    + resistor: 1 kOhm
    + diode: D1N4004

![Markdown-mark](../experiment/circuit_01.jpeg)

<!-- ![Markdown-mark](../experiment/measurement_01.png) -->

### 1. 실험 결과 파일을 읽는다.

In [ ]:
data = get_oscilloscpoe_result_tektronix('../experiment/measurement_01.csv')

### 2. 실험 결과에서 시간, 입력 전압, 출력 전압, 다이오드에 걸리는 전압을 가져온다.

측정치를 가져올 때 1000을 곱해 단위를 밀리로 변환한다.

In [ ]:
times = data['TIME'] * 1000
vins = data['Sample CH1'] * 1000
vouts = data['Sample CH2'] * 1000

다이오드에 걸리는 전압을 입력 전압과 출력 전압을 사용해 계산한다.

In [ ]:
vds = vins - vouts

### 3. 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
draw_plot(times, vds, 'V(VD)', 2)
plt.show()

### 4. 입력 전압, 출력 전압의 첨두값, 다이오드의 cut-in 전압, 다이오드에 걸리는 역방향 전압의 첨두값을 구한다.

In [ ]:
V_IN_peak_mV = vins.max()
V_OUT_peak_mV = vouts.max()
V_D_cut_in_mV = vds.max()
V_D_reverse_peak_mV = -vds.min()

print_value("V_IN_peak_mV", V_IN_peak_mV)
print_value("V_OUT_peak_mV", V_OUT_peak_mV)
print_value("V_D_cut_in_mV", V_D_cut_in_mV)
print_value("V_D_reverse_peak_mV", V_D_reverse_peak_mV)

### 5. 시간을 x축, 입력 전압과 출력 전압을 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.show()

### 6. 시간을 x축, 다이오드에 걸리는 전압을 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vds, 'V(VD)', 2)
plt.show()

*******************************************************************************
## 실험 2 결과 분석 (실험 과정 5)

실험 1의 회로의 입력 전압을 500 mV로 변경한다. 

<!-- ![Markdown-mark](../experiment/measurement_02.png) -->

### 1. 실험 결과 파일을 읽는다.

In [ ]:
data = get_oscilloscpoe_result_tektronix('../experiment/measurement_02.csv')

### 2. 실험 결과에서 시간, 입력 전압, 출력 전압, 다이오드에 걸리는 전압을 가져온다.

측정치를 가져올 때 1000을 곱해 단위를 밀리로 변환한다.

In [ ]:
times = data['TIME'] * 1000
vins = data['Sample CH1'] * 1000
vouts = data['Sample CH2'] * 1000

다이오드에 걸리는 전압을 입력 전압과 출력 전압을 사용해 계산한다.

In [ ]:
vds = vins - vouts

### 3. 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
draw_plot(times, vds, 'V(VD)', 2)
plt.show()

### 4. 입력 전압, 출력 전압의 첨두값을 구한다.

In [ ]:
V_IN_peak_mV = vins.max()
V_OUT_peak_mV = vouts.max()

print_value("V_IN_peak_mV", V_IN_peak_mV)
print_value("V_OUT_peak_mV", V_OUT_peak_mV)

### 5. 시간을 x축, 입력 전압과 출력 전압을 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.show()

### 6. 시간을 x축, 다이오드에 걸리는 전압을 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vds, 'V(VD)', 2)
plt.show()

*******************************************************************************
## 실험 3 결과 분석 (실험 과정 6)

### 회로도

* 사용한 소자
    + resistor: 1 kOhm
    + diode: D1N4004

![Markdown-mark](../experiment/circuit_03.jpeg)

<!-- ![Markdown-mark](../experiment/measurement_03.png) -->

### 1. 실험 결과 파일을 읽는다.

In [ ]:
data = get_oscilloscpoe_result_tektronix('../experiment/measurement_03.csv')

### 2. 실험 결과에서 시간, 입력 전압, 출력 전압, 다이오드에 걸리는 전압을 가져온다.

측정치를 가져올 때 1000을 곱해 단위를 밀리로 변환한다.

In [ ]:
times = data['TIME'] * 1000
vins = data['Sample CH1'] * 1000
vouts = data['Sample CH2'] * 1000

다이오드에 걸리는 전압을 입력 전압과 출력 전압을 사용해 계산한다.

In [ ]:
vds = vins - vouts

### 3. 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
draw_plot(times, vds, 'V(VD)', 2)
plt.show()

### 4. 입력 전압, 출력 전압의 첨두값을 구한다.

In [ ]:
V_IN_peak_mV = vins.max()
V_OUT_peak_mV = vouts.max()

print_value("V_IN_peak_mV", V_IN_peak_mV)
print_value("V_OUT_peak_mV", V_OUT_peak_mV)

### 5. 시간을 x축, 입력 전압과 출력 전압을 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.show()

### 6. 시간을 x축, 다이오드에 걸리는 전압을 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vds, 'V(VD)', 2)
plt.show()